In [3]:
!pip install pymongo

In [6]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.1/390.1 kB 7.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.8/982.8 kB 14.9 MB/s eta 0:00:00a 0:00:01


In [1]:
import pymongo
from pymongo import MongoClient

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["mydatabase"]
dbnames = client.list_database_names()
if "mydatabase" in dbnames:
    print("db exists. Will be deleted...")
    client.drop_database("mydatabase")


db exists. Will be deleted...


In [2]:
import json

In [3]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mydatabase')

In [4]:
with open('corona-out.json', 'r') as f:
    data = f.readlines()

In [5]:
from datetime import datetime

tweets_collection = db['tweets']
retweets_collection = db['retweets']

for record in data:
    try:
        tweet = json.loads(record.strip())
    except json.decoder.JSONDecodeError:
       # print(f'Error: Invalid JSON format in record "{record.strip()}". Skipping...')
        continue
    try:
        text = tweet['extended_tweet']['full_text']
    except KeyError:
        text = tweet['text']
    
    created_at = tweet['created_at']
    tweet_datetime = datetime.strptime(created_at, '%a %b %d %H:%M:%S %z %Y')
    month = tweet_datetime.strftime('%m')
    year = tweet_datetime.strftime('%Y')
    date = tweet_datetime.strftime('%d')


    # Extract relevant data from tweet JSON
    tweet_data = {
        'id': tweet['id'],
        'text': text,
        'month': month,
        'year': year,
        'date': date,
        'user_id': tweet['user']['id'],
        'user_name': tweet['user']['name'],
        'user_screen_name': tweet['user']['screen_name']
    }
    
    if 'retweeted_status' in tweet:
        try:
            retweet_text = tweet['retweeted_status']['extended_tweet']['full_text']
        except KeyError:
            retweet_text = tweet['retweeted_status']['text']
            
        retweet_data = {
            'rtwt_id': tweet['id'],
            'text': retweet_text,
            'month': month,
            'year': year,
            'date': date,
            'ruser_id': tweet['user']['id'],
            'ruser_name': tweet['user']['name'],
            'ruser_screen_name': tweet['user']['screen_name'],
            'otwt_id': tweet['retweeted_status']['id'],
            'ouser_id': tweet['retweeted_status']['user']['id'],
            'ouser_name': tweet['retweeted_status']['user']['name'],
            'ouser_screen_name': tweet['retweeted_status']['user']['screen_name']
        }
    
        
        # Insert retweet data into retweets collection
        retweets_collection.insert_one(retweet_data)
    else:
        
        # Insert tweet data into tweets collection
        tweets_collection.insert_one(tweet_data)


In [48]:
print(retweet_data)

{'rtwt_id': 1254059752201519104, 'text': 'För att förstå vad Anders Tegnell menar med ”ett snabbare förlopp” krävs att man förstår svaret på en gåta. Hur många gånger måste man vika ett vanligt A4-papper för att det till slut är så tjockt att det når ända till månen?\n\nBriljant genomgång 👇 https://t.co/VnrztqZIci', 'created_at': 'Sat Apr 25 14:48:38 +0000 2020', 'ruser_id': 933734367679123456, 'ruser_name': 'Jacob', 'ruser_screen_name': 'lundgreenjacob', 'otwt_id': 1253966779900137472, 'ouser_id': 19236255, 'ouser_name': 'Anders Lindberg', 'ouser_screen_name': 'anderslindberg'}


In [49]:
print(tweet_data)

{'id': 1254059752201519104, 'text': 'RT @anderslindberg: För att förstå vad Anders Tegnell menar med ”ett snabbare förlopp” krävs att man förstår svaret på en gåta. Hur många g…', 'created_at': 'Sat Apr 25 14:48:38 +0000 2020', 'user_id': 933734367679123456, 'user_name': 'Jacob', 'user_screen_name': 'lundgreenjacob'}


# Iterate over records and store in appropriate collection
import json
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['twitter_db']
tweets_collection = db['tweets_collection']
retweets_collection = db['retweets_collection']

# Read JSON file with exception handling
with open('twitter_data.json', 'r') as f:
    data = f.readlines()
    
for record in data:
    try:
        tweet = json.loads(record.strip())
    except json.decoder.JSONDecodeError:
        print(f'Error: Invalid JSON format in record "{record.strip()}". Skipping...')
        continue

    # Extract relevant data from tweet JSON
    try:
        text = tweet['extended_tweet']['full_text']
    except KeyError:
        text = tweet['text']

    tweet_data = {
        'id': tweet['id'],
        'text': text,
        'created_at': tweet['created_at'],
        'user': {
            'id': tweet['user']['id'],
            'name': tweet['user']['name'],
            'screen_name': tweet['user']['screen_name']
        }
    }

    # Check if tweet is a retweet and extract retweet data if so
    if 'retweeted_status' in tweet:
        try:
            retweet_text = tweet['retweeted_status']['extended_tweet']['full_text']
        except KeyError:
            retweet_text = tweet['retweeted_status']['text']
            
        retweet_data = {
            'id': tweet['id'],
            'text': retweet_text,
            'created_at': tweet['created_at'],
            'user': {
                'id': tweet['user']['id'],
                'name': tweet['user']['name'],
                'screen_name': tweet['user']['screen_name']
            },
            'retweeted_status': {
                'id': tweet['retweeted_status']['id'],
                'user': {
                    'id': tweet['retweeted_status']['user']['id'],
                    'name': tweet['retweeted_status']['user']['name'],
                    'screen_name': tweet['retweeted_status']['user']['screen_name']
                }
            }
        }
        
        # Insert retweet data into retweets collection
        retweets_collection.insert_one(retweet_data)
    else:
        # Insert tweet data into tweets collection
        tweets_collection.insert_one(tweet_data)
